Imports

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

Data

In [9]:
# Load the data
df = pd.read_csv('cleaned_data_water.csv')

# Create a list of all the US states in alphabetical order
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
             'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
             'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
             'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
             'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 
             'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
             'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

Training set and test set along with model for each state

In [ ]:
# Loop through each state
for state in us_states:
    # Filter data for a specific state
    df_state = df[df['State'] == state]

    # If df_state is empty, skip to the next iteration
    if df_state.empty:
        print(f"No data available for {state}, skipping.")
        continue

    # Convert 'ActivityStartDate' to datetime and extract the year, month, and day as integers
    df_state['ActivityStartDate'] = pd.to_datetime(df_state['ActivityStartDate'])
    df_state['Year'] = df_state['ActivityStartDate'].dt.year
    df_state['Month'] = df_state['ActivityStartDate'].dt.month
    df_state['Day'] = df_state['ActivityStartDate'].dt.day

    # Define the features and the target
    X = df_state[['Year', 'Month', 'Day']]
    y = df_state['ResultMeasureValue']

    # Normalize the features to be between 0 and 1
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    # Reshape input to be [samples, time steps, features]
    X_normalized = np.reshape(X_normalized, (X_normalized.shape[0], 1, X_normalized.shape[1]))

    # Split the data into training and testing sets (75% for training and 25% for testing)
    train_size = int(len(X_normalized) * 0.75)
    X_train, X_test = X_normalized[:train_size], X_normalized[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Define the model
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(4, input_shape=(1, 3)))
    model.add(tf.keras.layers.Dense(1))

    # Compile and fit the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=2)

    # Define the period for which we want a prediction
    future = np.array([[2024, i, 1] for i in range(1, 13)])  # 12 months in 2024
    future_normalized = scaler.transform(future)
    future_normalized = np.reshape(future_normalized, (future_normalized.shape[0], 1, future_normalized.shape[1]))

    # Use the model to make a prediction
    forecast = model.predict(future_normalized)
    # Save the model
    model.save('my_model'+state+'.h5')  # creates a HDF5 file 'my_model.h5' for corresponding state
    # Print the prediction
    print(f'Forecast for {state} in the future:')
    for i, y in enumerate(forecast):
        print(f'Month {i+1}: {y[0]}')
    
        



Predictions

In [10]:
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd



def predict_pollution(dates, state):
    # Load the model
    model = load_model('my_model'+state+'.h5')
    # Define the scaler
    scaler = MinMaxScaler()
    # Filter data for a specific state
    df_state = df[df['State'] == state]

    # Convert 'ActivityStartDate' to datetime and extract the year, month, and day as integers
    df_state['ActivityStartDate'] = pd.to_datetime(df_state['ActivityStartDate'])
    df_state['Year'] = df_state['ActivityStartDate'].dt.year
    df_state['Month'] = df_state['ActivityStartDate'].dt.month
    df_state['Day'] = df_state['ActivityStartDate'].dt.day

    # Define the features
    X = df_state[['Year', 'Month', 'Day']]

    # Fit the scaler
    scaler.fit(X)

    # Prepare the dates
    dates = pd.to_datetime(dates, format='%m/%Y')
    future = np.array([[date.year, date.month, 1] for date in dates])
    future_normalized = scaler.transform(future)
    future_normalized = np.reshape(future_normalized, (future_normalized.shape[0], 1, future_normalized.shape[1]))

    # Use the model to make a prediction
    forecast = model.predict(future_normalized)

    # Print the prediction
    for i, y in enumerate(forecast):
        print(f'Date: {dates[i].strftime("%m/%Y")}, Predicted Pollution: {y[0]}')

# Example usage:
predict_pollution(['05/2024', '02/2025', '03/2025'], 'California')


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2227768173.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['ActivityStartDate'] = pd.to_datetime(df_state['ActivityStartDate'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2227768173.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['ActivityStartDate'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2227768173.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

1/1 [==============================] - 0s 251ms/step
Date: 05/2024, Predicted Pollution: 96.06136322021484
Date: 02/2025, Predicted Pollution: 96.05938720703125
Date: 03/2025, Predicted Pollution: 96.06179809570312


Latest information

In [26]:
import pandas as pd
import numpy as np

# Load the data from the CSV file, skipping the first row if it is a header
df = pd.read_csv('cleaned_data_water.csv', skiprows=1, names=['State', 'Date', 'Data'])

# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# List of all US states
states = us_states

# List to store the most recent data for each state
most_recent_data_list = []

# Loop over all states
for state in states:
    # Filter the data for the current state
    state_data = df[df['State'] == state]

    # Check if state_data is not empty
    if not state_data.empty:
        # Sort the data by 'Date'
        state_data = state_data.sort_values('Date')

        # Get the most recent data for the current state
        most_recent_data_list.append(state_data.iloc[-1].to_dict())
    else:
        # If no data is found for the state, append the state name with null date and data
        most_recent_data_list.append({'State': state, 'Date': np.nan, 'Data': np.nan})

# Create a DataFrame from the list
most_recent_data = pd.DataFrame(most_recent_data_list)

# Print the most recent data for each state
print(most_recent_data)


             State       Date       Data
0          Alabama 2023-09-26     0.7600
1           Alaska 2023-08-23     0.0060
2          Arizona 2023-12-18     0.7000
3         Arkansas 2023-10-16     0.2600
4       California 2023-12-20     1.7000
5         Colorado 2023-12-11     7.1000
6      Connecticut 2023-12-14     1.4000
7         Delaware        NaT        NaN
8          Florida 2023-11-08    28.3000
9          Georgia 2023-12-11     2.0000
10          Hawaii 2023-08-02     0.0400
11           Idaho 2023-11-15    60.0000
12        Illinois 2023-12-20     5.6000
13         Indiana 2024-01-23    21.0000
14            Iowa 2023-10-24     0.8000
15          Kansas 2023-11-14     9.7000
16        Kentucky 2023-08-28     0.0400
17       Louisiana 2023-08-23     8.3600
18           Maine 2023-07-12     3.0000
19        Maryland 2023-12-21     5.9600
20   Massachusetts 2023-10-23     0.7000
21        Michigan 2023-10-24     0.1000
22       Minnesota 2023-10-11     0.2000
23     Mississip

For all states

In [20]:
print(df.columns)
print(df.head())


Index(['State', 'Date', 'Data'], dtype='object')
      State       Date     Data
0  Missouri 2022-09-13  11500.0
1  Missouri 2022-07-18     17.0
2  Missouri 2022-10-31      5.5
3  Missouri 2022-10-31      1.7
4  Missouri 2022-10-31      1.7


All States

In [27]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

def predict_pollution(dates, state):
    # Load the model
    if state.lower() == 'all':
        states = us_states
    else:
        states = [state]

    for state in states:
        model_path = 'my_model'+state+'.h5'
        if os.path.exists(model_path):
            model = load_model(model_path)
            # Define the scaler
            scaler = MinMaxScaler()
            # Filter data for a specific state
            df_state = df[df['State'] == state]

            # Convert 'Date' to datetime and extract the year, month, and day as integers
            df_state['Date'] = pd.to_datetime(df_state['Date'])
            df_state['Year'] = df_state['Date'].dt.year
            df_state['Month'] = df_state['Date'].dt.month
            df_state['Day'] = df_state['Date'].dt.day

            # Define the features
            X = df_state[['Year', 'Month', 'Day']]

            # Fit the scaler
            scaler.fit(X)

            # Prepare the dates
            dates = pd.to_datetime(dates, format='%m/%Y')
            future = np.array([[date.year, date.month, 1] for date in dates])
            future_normalized = scaler.transform(future)
            future_normalized = np.reshape(future_normalized, (future_normalized.shape[0], 1, future_normalized.shape[1]))

            # Use the model to make a prediction
            forecast = model.predict(future_normalized)

            # Print the prediction
            for i, y in enumerate(forecast):
                print(f'State: {state}, Date: {dates[i].strftime("%m/%Y")}, Predicted Pollution: {y[0]}')
        else:
            print(f'State: {state}, Date: N/A, Predicted Pollution: N/A')

            
predict_pollution(['02/2024', '03/2024', '04/2024','05/2024','06/2024','07/2024','08/2024','09/2024','10/2024','11/2024','12/2024','01/2025','02/2024'],'All')


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 194ms/step
State: Alabama, Date: 05/2024, Predicted Pollution: 20.139629364013672
State: Alabama, Date: 02/2025, Predicted Pollution: 19.916500091552734
State: Alabama, Date: 03/2025, Predicted Pollution: 20.009408950805664


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 179ms/step
State: Alaska, Date: 05/2024, Predicted Pollution: 58.53428649902344
State: Alaska, Date: 02/2025, Predicted Pollution: 58.46057891845703
State: Alaska, Date: 03/2025, Predicted Pollution: 58.50731658935547
1/1 [==============================] - ETA: 0s

C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

...

1/1 [==============================] - 0s 173ms/step
State: Washington, Date: 05/2024, Predicted Pollution: 30.24188232421875
State: Washington, Date: 02/2025, Predicted Pollution: 30.241943359375
State: Washington, Date: 03/2025, Predicted Pollution: 30.241943359375
State: West Virginia, Date: N/A, Predicted Pollution: N/A


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 178ms/step
State: Wisconsin, Date: 05/2024, Predicted Pollution: 18.972789764404297
State: Wisconsin, Date: 02/2025, Predicted Pollution: 18.94068145751953
State: Wisconsin, Date: 03/2025, Predicted Pollution: 18.957515716552734


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\3400172254.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 177ms/step
State: Wyoming, Date: 05/2024, Predicted Pollution: 50.35204315185547
State: Wyoming, Date: 02/2025, Predicted Pollution: 50.35204315185547
State: Wyoming, Date: 03/2025, Predicted Pollution: 50.35204315185547


Geojson file export

In [29]:
import os
import pandas as pd
import numpy as np
import json
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

def predict_pollution(dates, state):
    # Initialize an empty dictionary to store the predictions
    predictions = {}

    # Load the model
    if state.lower() == 'all':
        states = us_states
    else:
        states = [state]

    for state in states:
        model_path = 'my_model'+state+'.h5'
        if os.path.exists(model_path):
            model = load_model(model_path)
            # Define the scaler
            scaler = MinMaxScaler()
            # Filter data for a specific state
            df_state = df[df['State'] == state]

            # Convert 'Date' to datetime and extract the year, month, and day as integers
            df_state['Date'] = pd.to_datetime(df_state['Date'])
            df_state['Year'] = df_state['Date'].dt.year
            df_state['Month'] = df_state['Date'].dt.month
            df_state['Day'] = df_state['Date'].dt.day

            # Define the features
            X = df_state[['Year', 'Month', 'Day']]

            # Fit the scaler
            scaler.fit(X)

            # Prepare the dates
            dates = pd.to_datetime(dates, format='%m/%Y')
            future = np.array([[date.year, date.month, 1] for date in dates])
            future_normalized = scaler.transform(future)
            future_normalized = np.reshape(future_normalized, (future_normalized.shape[0], 1, future_normalized.shape[1]))

            # Use the model to make a prediction
            forecast = model.predict(future_normalized)

            # Store the prediction in the dictionary
            predictions[state] = {dates[i].strftime("%m/%Y"): float(y[0]) for i, y in enumerate(forecast)}
        else:
            predictions[state] = "N/A"

    # Convert the dictionary to a .geojson file
    with open('waterPredictions.geojson', 'w') as f:
        json.dump(predictions, f)

# Call the function
predict_pollution(['02/2024', '03/2024', '04/2024','05/2024','06/2024','07/2024','08/2024','09/2024','10/2024','11/2024','12/2024','01/2025','02/2024'],'All')


1/1 [==============================] - ETA: 0s

C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - ETA: 0s

C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - ETA: 0s

...

1/1 [==============================] - 0s 172ms/step


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 200ms/step


C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Date'] = pd.to_datetime(df_state['Date'])
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_state['Year'] = df_state['Date'].dt.year
C:\Users\Esteban\AppData\Local\Temp\ipykernel_121904\2575644805.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

1/1 [==============================] - 0s 191ms/step
